#### <strong>STEP 1: Load data and prepare for Torch</strong>
#### <strong>AT&T人臉圖示範</strong>

In [14]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
 
df = pd.read_csv('/Users/andrewhsu/Library/Mobile Documents/com~apple~CloudDocs/VsCode_python/Excel_file/face_data.csv')
n_persons = df['target'].nunique() 
X = np.array(df.drop('target', axis=1)) # 400 x 4096
y = np.array(df['target'])
 
# 以numpy的矩陣形式輸入，非dataframe
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size) # deafult test_size=0.25
 
# prepare data for PyTorch Tensor(一種矩陣資料型態)
X_train = torch.from_numpy(X_train).float() # convert to float tensor
y_train = torch.from_numpy(y_train).float() # 
train_dataset = TensorDataset(X_train, y_train) # create your datset # 再次把X、Y合起來
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()
test_dataset = TensorDataset(X_test, y_test) # create your datset
 
# create dataloader for PyTorch
# 一次讀取的資料量，不會整組拿去續練，而是一次拿一小部分
batch_size = 64 # 32, 64, 128, 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # convert to dataloader
test_loader = DataLoader(test_dataset, batch_size=len(X_test), shuffle=False)

#### <strong>STEP 2 :Set up NN Model</strong> 

In [22]:
import torch.nn as nn
import torch.nn.functional as F
 
# select device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = "cpu" # run faster than cuda in some cases
print("Using {} device".format(device))
 
# Create a neural network
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(64*64, 512), # image length 64x64=4096,  fully connected layer
            nn.ReLU(), # try to take ReLU out to see what happen
            nn.Linear(512, 128), # second hidden layer
            nn.ReLU(),
            nn.Linear(128, 40) # 40 classes,  fully connected layer
            # nn.Softmax()
        )
    # Specify how data will pass through this model
    def forward(self, x):
        # out = self.mlp(x) 
 
        # Apply softmax to x here~
        x = self.mlp(x)
        # 以下羅吉斯迴歸
        out = F.log_softmax(x, dim=1) # it’s faster and has better numerical propertie than softmax
        # out = F.softmax(x, dim=1)
        return out
 
 
# define model, optimizer, loss function
model = MLP().to(device) # start an instance
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # default lreaning rate=1e-3 #定義演算法
loss_fun = nn.CrossEntropyLoss() # define loss function #定義損失函數
 
print(model)

Using cpu device
MLP(
  (mlp): Sequential(
    (0): Linear(in_features=4096, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=40, bias=True)
  )
)


In [23]:
input = torch.randn(100, 64 * 64) 
m1 = nn.Linear(64*64, 512)
output = m1(input)
print(output.size())
output = F.relu(output)
print(output.size())
m2 = nn.Linear(512, 40)
output = m2(output)
print(output.size())
output = F.log_softmax(output, dim=1)
print(output.size())

output = F.softmax(output, dim=1)
print(output.size())
print(output)
print(output.argmax(dim=1))

torch.Size([100, 512])
torch.Size([100, 512])
torch.Size([100, 40])
torch.Size([100, 40])
torch.Size([100, 40])
tensor([[0.0257, 0.0222, 0.0231,  ..., 0.0209, 0.0186, 0.0311],
        [0.0203, 0.0202, 0.0231,  ..., 0.0175, 0.0239, 0.0217],
        [0.0297, 0.0179, 0.0184,  ..., 0.0275, 0.0180, 0.0241],
        ...,
        [0.0180, 0.0255, 0.0191,  ..., 0.0166, 0.0163, 0.0235],
        [0.0229, 0.0205, 0.0208,  ..., 0.0155, 0.0166, 0.0251],
        [0.0225, 0.0254, 0.0204,  ..., 0.0146, 0.0207, 0.0240]],
       grad_fn=<SoftmaxBackward0>)
tensor([31, 30,  9,  9, 28, 18,  6,  3,  9,  5,  5,  7, 36, 22, 28,  3, 32,  3,
        22, 32,  0,  9, 16, 15, 39,  5,  0,  9, 28, 28, 12, 22,  9, 28,  3, 18,
        22, 32, 18, 18,  9,  9, 30, 30,  0,  6, 20, 36,  8, 22, 28,  6,  9,  6,
        22, 22, 18,  6, 28, 30, 36, 22,  9, 21,  8, 22, 16, 36,  8, 17,  9, 32,
         6, 28,  7,  8,  6,  3, 33,  9,  6,  5, 28,  9, 25, 36, 36, 36,  3, 16,
        22, 31,  6, 28, 36, 22, 32, 28, 22, 31])


#### <strong>STEP 3: Start training</strong> 

In [24]:
from tqdm import tqdm
 
epochs = 50 # Repeat the whole dataset epochs times
model.train() # Sets the module in training mode. The training model allow the parameters to be updated during backpropagation.
for epoch in range(epochs):
# for epoch in tqdm(range(epochs)):
    trainAcc = 0
    samples = 0
    losses = []
    for batch_num, input_data in enumerate(train_loader):
    # for batch_num, input_data in tqdm(enumerate(train_loader), total=len(train_loader)):
         
        x, y = input_data
        x = x.to(device).float()
        y = y.to(device)
 
        # perform training based on the backpropagation
        y_pre = model(x) # predict y
        loss = loss_fun(y_pre, y.long()) # the loss function nn.CrossEntropyLoss()
        losses.append(loss.item())
 
        optimizer.zero_grad() # Zeros the gradients accumulated from the previous batch/step of the model
        loss.backward() # Performs backpropagation and calculates the gradients
        optimizer.step() # Updates the weights in our neural network based on the results of backpropagation
         
        # Record the training accuracy for each batch
        trainAcc += (y_pre.argmax(dim=1) == y).sum().item() # comparison
        samples += y.size(0)
        if batch_num % 4 == 0:
            print('\tEpoch %d | Batch %d | Loss %6.2f' % (epoch, batch_num, loss.item()))
    print('Epoch %d | Loss %6.2f | train accuracy %.4f' % (epoch, sum(losses)/len(losses), trainAcc/samples))
 
print('Finished ... Loss %7.4f | train accuracy %.4f' % (sum(losses)/len(losses), trainAcc/samples))


	Epoch 0 | Batch 0 | Loss   3.70
	Epoch 0 | Batch 4 | Loss   3.74
Epoch 0 | Loss   3.81 | train accuracy 0.0393
	Epoch 1 | Batch 0 | Loss   3.73
	Epoch 1 | Batch 4 | Loss   3.86
Epoch 1 | Loss   3.72 | train accuracy 0.0250
	Epoch 2 | Batch 0 | Loss   3.65
	Epoch 2 | Batch 4 | Loss   3.63
Epoch 2 | Loss   3.66 | train accuracy 0.0393
	Epoch 3 | Batch 0 | Loss   3.63
	Epoch 3 | Batch 4 | Loss   3.49
Epoch 3 | Loss   3.62 | train accuracy 0.0607
	Epoch 4 | Batch 0 | Loss   3.60
	Epoch 4 | Batch 4 | Loss   3.65
Epoch 4 | Loss   3.62 | train accuracy 0.0964
	Epoch 5 | Batch 0 | Loss   3.58
	Epoch 5 | Batch 4 | Loss   3.44
Epoch 5 | Loss   3.56 | train accuracy 0.0643
	Epoch 6 | Batch 0 | Loss   3.52
	Epoch 6 | Batch 4 | Loss   3.66
Epoch 6 | Loss   3.56 | train accuracy 0.0964
	Epoch 7 | Batch 0 | Loss   3.49
	Epoch 7 | Batch 4 | Loss   3.46
Epoch 7 | Loss   3.50 | train accuracy 0.0786
	Epoch 8 | Batch 0 | Loss   3.46
	Epoch 8 | Batch 4 | Loss   3.40
Epoch 8 | Loss   3.44 | train accuracy

#### <strong>Testing (1) : Compute test accuracy by batch</strong> 
完成訓練後的神經網路模組，必須透過測試資料進行測試

In [25]:
model.eval() 
testAcc = 0
samples = 0
with torch.no_grad():
    for x, y_truth in test_loader:
        x = x.to(device).float()
        y_truth = y_truth.to(device)
        y_pre = model(x).argmax(dim=1) # the predictions for the batch
        testAcc += (y_pre == y_truth).sum().item() # comparison
        samples += y_truth.size(0)
 
    print('Test Accuracy:{:.3f}'.format(testAcc/samples))

Test Accuracy:0.758


#### <strong>Testing (2): Compute the test accuracy and record the result for each test data</strong> 

In [26]:
import csv
 
# use eval() in conjunction with a torch.no_grad() context, 
# meaning that gradient computation is turned off in evaluation mode
model.eval() 
testAcc = 0
samples = 0
 
with open('mlp_att.csv', 'w') as f:
    fieldnames = ['ImageId', 'Label', 'Ground_Truth']
    writer = csv.DictWriter(f, fieldnames=fieldnames, lineterminator = '\n')
    writer.writeheader()
    image_id = 1
 
    with torch.no_grad():
        for x, y_truth in test_loader:
            x = x.to(device).float()
            y_truth = y_truth.to(device).long()
            yIdx = 0
            y_pre = model(x).argmax(dim=1) # the predictions for the batch
            testAcc += (y_pre == y_truth).sum().item() # comparison
            samples += y_truth.size(0)
            for y in y_pre:
                writer.writerow({fieldnames[0]: image_id,fieldnames[1]: y.item(), fieldnames[2]: y_truth[yIdx].item()})
                image_id += 1
                yIdx += 1
 
        print('Test Accuracy:{:.3f}'.format(testAcc/samples))

Test Accuracy:0.758
